In [2]:
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## CLEANING IMDb_Movies 

#### Read in IMDb_Movies Table

In [7]:
movie_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Movies.csv")

In [25]:
movie_df.head()

,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID
0,17761,The Shawshank Redemption,Frank Darabont,R,Drama,142,1994,43,21,7041
1,17762,The Godfather,Francis Ford Coppola,R,"Crime, Drama",175,1972,30,32,7042
2,17763,The Dark Knight,Christopher Nolan,PG-13,"Action, Crime, Drama",152,2008,163,160,7043
3,17764,The Godfather Part II,Francis Ford Coppola,R,"Crime, Drama",202,1974,21,17,7044
4,17765,12 Angry Men,Sidney Lumet,Approved,"Crime, Drama",96,1957,13,17,7045


In [29]:
movie_df[0:21]

,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID
0,17761,The Shawshank Redemption,Frank Darabont,R,Drama,142,1994,43,21,7041
1,17762,The Godfather,Francis Ford Coppola,R,"Crime, Drama",175,1972,30,32,7042
2,17763,The Dark Knight,Christopher Nolan,PG-13,"Action, Crime, Drama",152,2008,163,160,7043
3,17764,The Godfather Part II,Francis Ford Coppola,R,"Crime, Drama",202,1974,21,17,7044
4,17765,12 Angry Men,Sidney Lumet,Approved,"Crime, Drama",96,1957,13,17,7045
5,17766,Schindler's List,Steven Spielberg,R,"Biography, Drama, History",195,1993,49,91,7046
6,17767,The Lord of the Rings: The Return of the King,Peter Jackson,PG-13,"Action, Adventure, Drama",201,2003,124,209,7047
7,17768,Pulp Fiction,Quentin Tarantino,R,"Crime, Drama",154,1994,75,70,7048
8,17769,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,PG-13,"Action, Adventure, Drama",178,2001,126,121,7049
9,177610,"The Good, the Bad and the Ugly",Sergio Leone,R,"Adventure, Western",178,1966,6,3,70410


##### Drop 'Unnamed: 0' which I believe is the index in the csv being imported in

In [13]:
movie_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [15]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title_ID      250 non-null    int64 
 1   Title         250 non-null    object
 2   Director      250 non-null    object
 3   Rating        250 non-null    object
 4   Genre         250 non-null    object
 5   Runtime       250 non-null    int64 
 6   Release_Year  250 non-null    int64 
 7   Award_Noms    250 non-null    int64 
 8   Award_Wins    250 non-null    int64 
 9   Score_ID      250 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 19.7+ KB


#### Next, I need to clean up Ratings and the discrepencies there

In [ ]:
Source:
Link
Summary:

Source:
Link
Summary:
    
Source:
Link
Summary:

In [ ]:
From the research that I have conducted, I have decided that this will be the standard to limit the discrepencies in the rating categories.
    DECISION: 

In [22]:
movie_df['Rating'].value_counts()

R                        101
PG                        37
PG-13                     34
Not Rated                 24
G                         19
Passed                    16
Approved                  13
Not Available on IMDb      1
TV-PG                      1
Unrated                    1
X                          1
TV-MA                      1
GP                         1
Name: Rating, dtype: int64

## CLEANING IMDb_Score

#### Read in IMDb_Score Table

In [11]:
score_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Score.csv")

In [12]:
score_df.head()

,Unnamed: 0,Score_ID,IMDb_Score,Audience_Reviews,US_Users,Non_US_Users,Male_Reviews,Female_Reviews,Male_Score,Female_Score
0,0,7041,9.3,2666904,566032,1672983,1475251,294889,9.3,9.2
1,1,7042,9.2,1848055,417566,1161734,1072921,166441,9.2,8.9
2,2,7043,9.0,2639814,621027,1646769,1490958,267216,9.0,8.7
3,3,7044,9.0,1265636,282291,805848,759883,105690,9.0,8.7
4,4,7045,9.0,787589,187528,509017,453568,76860,9.0,8.9


Drop first column

In [16]:
score_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [17]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Score_ID          250 non-null    int64  
 1   IMDb_Score        250 non-null    float64
 2   Audience_Reviews  250 non-null    int64  
 3   US_Users          250 non-null    int64  
 4   Non_US_Users      250 non-null    int64  
 5   Male_Reviews      250 non-null    int64  
 6   Female_Reviews    250 non-null    int64  
 7   Male_Score        250 non-null    float64
 8   Female_Score      250 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 17.7 KB


### with cleaned up tables, I need to save them and manage them

#### Saving as CSV for safe measure

In [ ]:
TBD

#### ETL: Push tables directly to Postgres SQL database (instead of manually downloading csv files)

In [ ]:
TBD